In [ ]:
!pip3 install python-sat

In [ ]:
import numpy as np
import time
from random import randrange
from random import seed
from random import shuffle
from pysat.solvers import Glucose4
from pysat.solvers import Minisat22
from pysat.solvers import MapleCM
from pysat.solvers import Lingeling
from pysat.solvers import Minicard
import matplotlib.pyplot as plt
import math
import networkx as nx
import random

In [ ]:
def print_grid(grid, m):
    count = 0
    for elem in grid:
        print(elem, end=" ")
        count = (count + 1)%m
        if count==0:
            print()
    print()

class Graph:
    def __init__(self, n):
        self.n = n
        self.adj = [[] for i in range(n)]
        self.degree = 0    #maximum degree

    def add_edge(self, u, v):
        self.adj[u].append(v)
        self.adj[v].append(u)
        if len(self.adj[u])>self.degree or len(self.adj[v])>self.degree:
            self.degree+=1

    # Find an arbitrary coloring

    def find_coloring(self, q):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            tmp = [0 for _ in range(q)]
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    tmp[colors[u]] = 1
            for it in range(q):
                if tmp[it] == 0:
                    colors[i] = it
                    break
        return colors


    def find_coloring_cons(self, q, border, border_colors):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            blocked = [0 for _ in range(q)]
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    blocked[colors[u]] = 1
            if i in border:
                for clr in range(q):
                    if clr not in border_colors[i]:
                        blocked[clr] = 1
            for it in range(q):
                if blocked[it] == 0:
                    colors[i] = it
                    break
        return colors



    def find_different_coloring(self, q):
        colors = [-1 for _ in range(self.n)]
        for i in range(self.n):
            available = [True for _ in range(q)]
            available[0] = False
            for u in self.adj[i]:
                if (colors[u] == -1):
                    continue
                else:
                    available[colors[u]] = False
            for it in range(q):
                if available[it]:
                    colors[i] = it
                    break
        return colors



    '''
        Forward simulation of the Glauber dynamics;
            coloring: some arbitrary (valid) coloring,
            transitions: a list of (node, color) pairs
    '''

    #Simplify transitions by using bounding lists
    def fix_transitions(self, transitions, q):

        transitions_new = []
        bounding_lists = [set([clr for clr in range(q)]) for _ in range(self.n)]
        for (node, colors) in reversed(transitions):
            #there are some colors that could be blocked or not (maybe_blocked)
            blocked = set()
            available = set([clr for clr in range(q)])
            for u in self.adj[node]:
                if len(bounding_lists[u])==1:
                    for clr in bounding_lists[u]:
                        blocked.add(clr)
                for clr in bounding_lists[u]:
                    available.discard(clr)

            colors_new = []
            for color in colors:
                if color in blocked:
                    continue
                colors_new.append(color)
                if color in available:
                    break

            bounding_lists[node]=set(colors_new)
            transitions_new.append((node,colors_new))

        transitions_new.reverse()
        return transitions_new


    def glauber_dynamics(self, coloring, transitions):
        states = []

        for (node, color) in reversed(transitions):

            flag = True

            for u in self.adj[node]:
                if (coloring[u] == color):
                    flag = False

            if flag:
                coloring[node] = color

            states.append(flag)

        return coloring, states


    #Each transition is of the form (node, list of (degree+1) colors)
    def glauber_dynamics_perm(self, coloring, transitions):

        for (node, colors) in reversed(transitions):

            for color in colors:
                blocked = False
                for u in self.adj[node]:
                    if (coloring[u] == color):
                        blocked = True

                if not blocked:
                    coloring[node] = color
                    break

        return coloring

    # Return a set 'fam' of vertices with distance at most R from v
    # and a set 'brdr' of all vertices in 'fam' with neighbors outside of 'fam'.

    def family(self, v, R):

        visited = {v}   # Set of visited nodes.
        queue = [v]     # Initialize a queue

        for _ in range(R):
            new_queue = []
            for u in queue:
                for neigh in self.adj[u]:
                    if neigh not in visited:
                        visited.add(neigh)
                        new_queue.append(neigh)
            queue = new_queue.copy()

        border = set()
        for u in queue:
            for neigh in self.adj[u]:
                if neigh not in visited:
                    border.add(u)
                    break

        return visited, border


In [ ]:
print([set([clr for clr in range(4)]) for _ in range(3)])

[{0, 1, 2, 3}, {0, 1, 2, 3}, {0, 1, 2, 3}]


In [ ]:
#checks whether a collapse has occured
def CFTPcollapse_permutation(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(n):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation(graph, q, solver_name):

    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = randrange(n)
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        ret_value = CFTPcollapse_permutation(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")


        print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
#checks whether a collapse has occured for a single vertex
def CFTPcollapse_permutation2(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(1):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation2(graph, q, solver_name, block):

    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = block[len(transitions)%len(block)]   #systematic scan in block
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        #transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        ret_value = CFTPcollapse_permutation2(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            #print("Before:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")
            #print("After:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)


        print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
#Using "DP" instead of "Divide and Conquer"
def CFTPcollapse_permutation6(graph, q, solver_name, transitions, border, border_colors, color0):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    ##init_state = graph.find_coloring_cons(q, border, border_colors)
    #print_grid(init_state, m)
    #print("primary coloring:", init_state)
    ##final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # Enforce border available colors
    for u in border:
        for clr in range(q):
            if clr not in border_colors[u]:
                g.add_clause([-S(u, clr)])


    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    ##clause = []

    ##for i in range(1):
    ##    if (d[i] == -1):
    ##        clause.append(-S(i, final_state[i]))
     ##   else:
    ##        clause.append(-P(d[i], final_state[i]))

    ##g.add_clause(clause)

    if (d[0] == -1):
        g.add_clause([S(0, color0)])
    else:
        g.add_clause([P(d[0], color0)])

    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return coloring

    print('INFEASIBLE')
    return None

#entropy = number of possible colors (chosen randomly) for each vertex of border
def CFTPsolver_permutation6(graph, q, solver_name, block, border, entropy_dist):

    ret_value = 1
    transitions = []
    t = 1

    border_colors={}
    for u in border:
        #choose entropy from entropy_dist
        thr = random.uniform(0, 1)
        sum = 0
        for i in range(len(entropy_dist)):
            sum += entropy_dist[i]
            if sum >= thr:
                break
        entropy = i
        print(entropy)

        #choose random restrictions on border
        colors = [clr for clr in range(q)]

        #to make sure vertices with 1 option wont be blocked by neighbours
        if entropy == 1:
            for neigh in graph.adj[u]:
                if neigh in border_colors and len(border_colors[neigh])==1:
                    colors.remove(list(border_colors[neigh])[0])
        shuffle(colors)
        border_colors[u]=set(colors[:entropy])
    print(border, border_colors)

    possible_colors = set([clr for clr in range(q)])

    while(True):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = block[len(transitions)%len(block)]   #systematic scan in block
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        #transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        new_possible_colors = possible_colors.copy()
        for color0 in possible_colors:
            print("color:", color0)
            ret_value = CFTPcollapse_permutation6(graph, q, solver_name, transitions, border, border_colors, color0)
            print(time.time() - time_before)
            time_before = time.time()
            if ret_value is None:
                new_possible_colors.remove(color0)
            #else:
                #print_grid(ret_value,m)
        print(new_possible_colors)
        print(transitions[0])
        if len(new_possible_colors)<=1:
            break
        possible_colors = new_possible_colors

        ##if ret_value is not None:
            ##coloring1, coloring2 = ret_value
            #print("Before:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)
            ##coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            ##coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            ##print(coloring1 == coloring2, "should be False")
            #print("After:")
            #print_grid(coloring1,m)
            #print_grid(coloring2,m)
            ##break


        #print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
border_colors={}
border=[3,5,6]
entropy=3
q=7
for u in border:
    colors = [clr for clr in range(q)]
    shuffle(colors)
    border_colors[u]=set(colors[:entropy])
print(border_colors)

{3: {4, 5, 6}, 5: {0, 3, 4}, 6: {3, 4, 5}}


In [ ]:
#the color of some vertices is given
def CFTPcollapse_permutation4(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1

    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    for i in range(1,4):
        init_state[i] = i%(q-2)+2
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # fix given colors
    for i in range(1,4):
        g.add_clause([S(i, i%(q-2)+2)])

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(1):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation4(graph, q, solver_name, block):

    ret_value = 1
    transitions = []
    t = 1
    print(block)
    block = [0] + block[4:]
    print(block)
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = block[len(transitions)%len(block)]   #systematic scan in block
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        #transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        ret_value = CFTPcollapse_permutation4(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")


        print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
#burn-in: max number of different color neighbours in initial coloring
def CFTPcollapse_permutation5(graph, q, solver_name, transitions):

    n = graph.n
    t = len(transitions)

    # Functions that given the name of a variable returns its index
    #initial color of vertex i
    def S(i, clr):
        return i*q + clr + 1
    #color after transition k
    def P(k, clr):
        return n*q + k*q + clr + 1
    # d[i] is the last time vertex i was touched
    d = {i : -1 for i in range(n)}

    # Determine some final state
    init_state = graph.find_coloring(q)
    #print("primary coloring:", init_state)
    final_state = graph.glauber_dynamics_perm(init_state.copy(), transitions)
    #print("final state:", final_state)

    if solver_name == "glucose":
        g = Glucose4()
    elif solver_name == "lingeling":
        g = Lingeling()
    elif solver_name == "minisat":
        g = Minisat22()
    elif solver_name == "mapleCM":
        g = MapleCM()
    elif solver_name == "minicard":
        g = Minicard()

    # For each s_i exactly one color is set to 1 while the others are set to 0
    for i in range(n):
        # At least one color is set to 1
        g.add_clause([S(i, clr) for clr in range(q)])

        # At most one color is set to 1
        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-S(i, clr1), -S(i, clr2)])


    # The initial coloring has to be valid
    for u in range(n):
        for v in graph.adj[u]:
            for clr in range(q):
                g.add_clause([-S(u, clr), -S(v, clr)])


    # For each p_i exactly one color is set to 1 while the others are set to 0
    for k in range(t):
        g.add_clause([P(k, clr) for clr in range(q)])

        for clr1 in range(q):
            for clr2 in range(clr1+1, q):
                g.add_clause([-P(k, clr1), -P(k, clr2)])


    for k in range(t):

        node, colors = transitions[-k-1] # apply the transitions in reverse order

        for clr in range(q):
            if clr not in colors:
                g.add_clause([-P(k, clr)])

        for index in range(len(colors)):
            color = colors[index]

            # If S(u,color)=1 for some u, it means that u is a neighbour of
            # 'node' with color 'color'. In that case P(k,color) is set to 0
            for u in graph.adj[node]:
                if (d[u] == -1):
                    g.add_clause([-P(k,color), -S(u,color)])
                else:
                    g.add_clause([-P(k,color), -P(d[u],color)])

            #It is also set to 0 if any previous P(k,color) has been set to 1,
            #but we ensured this already.

            # If S(u,color)=0 for all u and all previous P(k,color) have been set to 0,
            #then P(k,color) is set to 1
            clause = []

            clause.append(P(k,color))

            for u in graph.adj[node]:
                if (d[u] == -1):
                    clause.append(S(u,color))
                else:
                    clause.append(P(d[u],color))

            for prev in range(index):
                clause.append(P(k,colors[prev]))

            g.add_clause(clause)

            d[node] = k


    # The final state is different than that of the master
    clause = []

    for i in range(n):
        if (d[i] == -1):
            clause.append(-S(i, final_state[i]))
        else:
            clause.append(-P(d[i], final_state[i]))

    g.add_clause(clause)


    # Solve the CSP
    status = g.solve()

    if (status):
        print('FEASIBLE')
        model = g.get_model()
        coloring = [-1 for _ in range(n)]

        for u in range(n):
            for clr in range(q):
                if model[S(u, clr)-1]>0:
                    coloring[u] = clr

        traj = [-1 for _ in range(t)]
        for k in range(t):
            for clr in range(q):
                if model[P(k, clr)-1]>0:
                    traj[k] = clr

        #print(model)
        #print("secondary coloring:", coloring)
        #print(traj)
        #print(transitions)
        return init_state, coloring

    print('INFEASIBLE')
    return None


#possible improvement: degree of each specific vertex instead of max degree
def CFTPsolver_permutation5(graph, q, solver_name):

    ret_value = 1
    transitions = []
    t = 1
    while(ret_value is not None):
        while(len(transitions)<t):
            # Select a random "Glauber" transition
            node = randrange(n)
            colors = [clr for clr in range(q)]
            shuffle(colors)
            trans = (node, colors[:(graph.degree+1)])
            transitions.append(trans)

        time_before = time.time()

        print(t)
        #coloring1 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(transitions)
        transitions = graph.fix_transitions(transitions, q)
        #coloring2 = graph.glauber_dynamics_perm(graph.find_coloring(q), transitions)
        #print(coloring1==coloring2, "should be True")
        #print(transitions)

        ret_value = CFTPcollapse_permutation(graph, q, solver_name, transitions)

        if ret_value is not None:
            coloring1, coloring2 = ret_value
            coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
            coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
            print(coloring1 == coloring2, "should be False")


        print(time.time() - time_before)
        t*=2

    return transitions

In [ ]:
# Construct an m x m square grid

m = 24

n = m*m

graph = Graph(n)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)

In [ ]:
#Construct a periodic m x m grid

m = 25

n = m*m

graph = Graph(n)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*j+i, m*j+i+1)

for j in range(m):
    for i in range(m-1):
        graph.add_edge(m*i + j, m*i+m + j)

for j in range(m):
    graph.add_edge(m*j, m*j+m-1)

for i in range(m):
    graph.add_edge(i, m*(m-1) + i)


In [ ]:
R = 7
block = []
border = []
for j in range(m):
    for i in range(m):
        if (i<=R or i>=m-R) and (j<=R or j>=m-R):
            block.append(m*j+i)
        if ((i==R+1 or i==m-R-1) and (j<=R or j>=m-R)) or ((i<=R or i>=m-R) and (j==R+1 or j==m-R-1)):
            border.append(m*j+i)

#print(block)
print(len(block))
#print(border)
print(len(border))



225
60


In [ ]:
#total number of edges

sumi = 0
for i in range(n):
    sumi += len(graph.adj[i])
print(sumi//2)

3958


In [ ]:
print(graph.degree)

9


In [ ]:
#add random edges

for i in range(n):
    for j in range(n):
        dist = abs(i%m - j%m) + abs(i//m - j//m)
        a = random.uniform(0, 1)
        if i<j and dist >= 2 and a <= 3.6**(-dist):
            graph.add_edge(i, j)



In [ ]:
R = 8
block, _ = graph.family(0, R)
block = list(block)
print(block)
print(len(block))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 1568, 1240, 236, 237, 238, 239, 240, 241, 242, 243, 1280, 1281, 31, 277, 278, 279, 280, 281, 282, 1319, 1320, 1321, 1322, 318, 319, 320, 321, 1358, 1359, 1360, 1361, 1362, 1363, 359, 360, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527]
181


In [ ]:
print(graph.adj[45])

[44, 46, 5, 85, 4, 48]


In [ ]:
R = 7
block = []
for j in range(m):
    for i in range(m):
        if (i<=R or i>=m-R) and (j<=R or j>=m-R) and (m*j+i==0 or m*j+i>=15):
            block.append(m*j+i)

print(len(block))


218


In [ ]:
m = 10
n = m*m*m
graph = Graph(n)
G = nx.grid_graph(dim=(m, m, m), periodic = True)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(map[node], map[neigh])
        graph.add_edge(map[node], map[neigh])

Graph with 1000 nodes and 3000 edges


In [ ]:
R = 4
block = []
for j in range(m):
    for i in range(m):
        for k in range(m):
            if (i<=R or i>=m-R) and (j<=R or j>=m-R) and (k<=R or k>=m-R):
                block.append(map[(j,i,k)])

print(len(block))


729


In [ ]:
n = 1000
d = 10
R = 5
graph = Graph(n)
G = nx.random_regular_graph(d, n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(map[node], map[neigh])
        graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block, len(block))


Graph with 1000 nodes and 5000 edges
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 21

In [ ]:
R = 14
m=40
n=60

G = nx.triangular_lattice_graph(m, n, periodic = True)
n = G.number_of_nodes()
graph = Graph(n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(node, neigh)
        if map[node]>map[neigh]:
            graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block)
print(len(block))

Graph with 1200 nodes and 3600 edges
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 2

In [ ]:
m=22
n=22
R = 14

G = nx.hexagonal_lattice_graph(m, n, periodic = True)
n = G.number_of_nodes()
graph = Graph(n)

map = {}
i=0
for node in G.nodes():
    map[node] = i
    i+=1

print(G)
for node in G.nodes():
    for neigh in G[node]:
        #print(node, neigh)
        graph.add_edge(map[node], map[neigh])

block, _ = graph.family(0, R)
block = list(block)
print(block, len(block))

Graph with 968 nodes and 1452 edges
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 345, 346, 347, 348, 349, 350, 351, 661, 663, 665, 667, 697, 699, 701, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 7

In [ ]:

seed(17)

ret_value = 1
transitions = []
t = 1
q = 6
while(True):
    while(len(transitions)<t):
        # Select a random "permutation" transition
        node = block[len(transitions)%len(block)]   #systematic scan
        color = randrange(q)
        trans = (node, color)
        transitions.append(trans)

    time_before = time.time()
    print(t)
    coloring1 = graph.find_coloring(q)
    coloring2 = graph.find_different_coloring(q)
    coloring1, _ = graph.glauber_dynamics(coloring1, transitions)
    coloring2, _ = graph.glauber_dynamics(coloring2, transitions)
    print_grid(coloring1,m)
    print_grid(coloring2,m)
    same = 0
    for i in range(len(coloring1)):
        if (coloring1[i]==coloring2[i]):
            same += 1
    print(same/len(coloring1))
    if coloring1[0] == coloring2[0]:
        break
    t*=2

In [ ]:
seed(16)

ret_value = 1
transitions = []
t = 1
q = 7
while(True):
    while(len(transitions)<t):
        # Select a random "permutation" transition
        node = block[len(transitions)%len(block)]   #systematic scan
        colors = [clr for clr in range(q)]
        shuffle(colors)
        trans = (node, colors[:(graph.degree+1)])
        transitions.append(trans)


    time_before = time.time()
    print(t)
    coloring1 = graph.find_coloring(q)
    coloring2 = graph.find_different_coloring(q)
    for i in range(n):
        coloring2[i]=1-coloring1[i]
    coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
    coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
    print_grid(coloring1,m)
    print_grid(coloring2,m)
    same = 0
    for i in range(len(coloring1)):
        if (coloring1[i]==coloring2[i]):
            same += 1
    print(same/len(coloring1))
    if coloring1 == coloring2:
        break
    t*=2

In [ ]:
seed(16)

ret_value = 1
transitions = []
t = 1
q = 5
while(True):
    while(len(transitions)<t):
        # Select a random "permutation" transition
        node = block[len(transitions)%len(block)]   #systematic scan
        colors = [clr for clr in range(q)]
        shuffle(colors)
        trans = (node, colors[:(graph.degree+1)])
        transitions.append(trans)


    time_before = time.time()
    print(t)
    coloring1 = graph.find_coloring(q)
    coloring = []
    for clr in range(q):
        new_coloring = coloring1.copy()
        for i in range(n):
            new_coloring[i]=(coloring1[i]+clr)%q
        coloring.append(new_coloring)
    for clr in range(q):
        #print_grid(coloring[clr],m)
        coloring[clr] = graph.glauber_dynamics_perm(coloring[clr], transitions)
        #print_grid(coloring[clr],m)
    same = True
    for clr in range(q):
        same = same and coloring[clr][0] == coloring[(clr+1)%q][0]
    if same:
        break
    t*=2

for clr in range(q):
    print(clr)
    print_grid(coloring[clr],m)

In [ ]:
q=7

t=10000
transitions = []
while(len(transitions)<t):
    # Select a random "Glauber" transition
    node = randrange(n)
    colors = [clr for clr in range(q)]
    shuffle(colors)
    trans = (node, colors[:(graph.degree+1)])
    transitions.append(trans)

init_state = graph.find_coloring(q)
print_grid(init_state, m)
final_state = graph.glauber_dynamics_perm(init_state, transitions)
print(transitions)
print_grid(final_state, m)

In [ ]:
for seednum in range(11,12):

    seed(seednum)

    q = 10 # Number of colors

    solvers = ["glucose"]

    lst_x_axis = []
    lst_stamps = []

    for solver_name in solvers:

        transitions = CFTPsolver_permutation2(graph, q, solver_name, block)

        coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
        coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
        coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
        print(transitions)
        print(coloring1[0] == coloring2[0], "should be True")

In [ ]:
for seednum in range(2,3):

    seed(seednum)

    q = 6 # Number of colors
    entropy_dist = [0,0,0,0,1]

    solvers = ["glucose"]

    for solver_name in solvers:

        transitions = CFTPsolver_permutation6(graph, q, solver_name, block, border, entropy_dist)

        coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
        coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
        coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
        print(transitions)
        print(coloring1[0] == coloring2[0], "should be True")

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
[8, 17, 33, 42, 58, 67, 83, 92, 108, 117, 133, 142, 158, 167, 183, 192, 200, 201, 202, 203, 204, 205, 206, 207, 218, 219, 220, 221, 222, 223, 224, 425, 426, 427, 428, 429, 430, 431, 432, 443, 444, 445, 446, 447, 448, 449, 458, 467, 483, 492, 508, 517, 533, 542, 558, 567, 583, 592, 608, 617] {8: {1, 2, 3, 4}, 17: {0, 1, 3, 4}, 33: {0, 2, 3, 5}, 42: {0, 1, 3, 5}, 58: {1, 3, 4, 5}, 67: {0, 2, 3, 5}, 83: {0, 3, 4, 5}, 92: {0, 1, 2, 3}, 108: {0, 1, 3, 5}, 117: {0, 1, 2, 4}, 133: {0, 1, 2, 3}, 142: {0, 1, 4, 5}, 158: {0, 2, 3, 4}, 167: {0, 1, 2, 3}, 183: {0, 1, 3, 5}, 192: {2, 3, 4, 5}, 200: {2, 3, 4, 5}, 201: {1, 2, 4, 5}, 202: {2, 3, 4, 5}, 203: {1, 2, 3, 4}, 204: {0, 2, 3, 4}, 205: {0, 2, 3, 4}, 206: {1, 2, 3, 4}, 207: {0, 1, 2, 5}, 218: {0, 2, 3, 4}, 219: {1, 3, 4, 5}, 220: {0, 1, 2, 4}, 221: {0, 1, 2, 5}, 222: {1, 3, 4, 5}, 223: {0, 2, 3, 4}, 224: {1, 3, 4, 5}, 425: {0

error: Caught keyboard interrupt

In [ ]:
for seednum in range(6, 26):

    seed(seednum)

    q = 7 # Number of colors

    solvers = ["glucose"]

    lst_x_axis = []
    lst_stamps = []

    for solver_name in solvers:

        transitions = CFTPsolver_permutation4(graph, q, solver_name, block)

        coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
        coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
        coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
        print(transitions)
        print(coloring1[0] == coloring2[0], "should be True")

In [ ]:
for seednum in range(1,5):

    seed(seednum)

    q = 6 # Number of colors

    solvers = ["glucose"]

    lst_x_axis = []
    lst_stamps = []

    v1=0
    v2=52

    for solver_name in solvers:

        transitions = CFTPsolver_permutation3(graph, q, solver_name, block, v1, v2)

        coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
        coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
        coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
        print(transitions)
        print(coloring1, coloring2)
        print(coloring1[v1], coloring1[v2], coloring2[v1], coloring2[v2])

In [ ]:
q = 7 # Number of colors

seed(16)

solvers = ["glucose"]

lst_x_axis = []
lst_stamps = []

for solver_name in solvers:

    transitions = CFTPsolver_permutation(graph, q, solver_name)

    coloring1, coloring2 = graph.find_coloring(q), graph.find_different_coloring(q)
    coloring1 = graph.glauber_dynamics_perm(coloring1, transitions)
    coloring2 = graph.glauber_dynamics_perm(coloring2, transitions)
    #print(transitions)
    print(coloring1 == coloring2, "should be True")



In [ ]:
print(graph.adj)

In [ ]:
q = 15
t=500000
transitions = []
while(len(transitions)<t):
    # Select a random "Glauber" transition
    node = randrange(n)
    colors = [clr for clr in range(q)]
    shuffle(colors)
    trans = (node, colors[:(graph.degree+1)])
    transitions.append(trans)

l = graph.fix_transitions(transitions, q)
print(transitions)
print(l)
for i in range(100):
  print(len(l[i][1]))
